In [ ]:
import xarray as xr
import netCDF4 as nc
import numpy as np
import glob
import os
import wrf
import matplotlib.pyplot as plt
import warnings
# import seaborn as sns

warnings.filterwarnings('ignore')

In [ ]:
#создание списка из названий файлов
filedir = '../Data/originals/'
filename = []
filename += glob.glob(filedir+os.sep+'*_cut.nc')

filename.sort()

In [ ]:
#сохраняем все возможные таймстепы
times = []
for file in filename:
    df = nc.Dataset(file,engine='netcdf4')
    if file[34:36] not in ['01', '02', '03', '04', '05', '10', '11', '12']:
        df.close()
        continue
    a = wrf.getvar(df, 'times', timeidx=wrf.ALL_TIMES)
#     print(a)
    times.append(a)

np.asarray(times).shape
np.save(arr = times, file ='times.npy')

In [ ]:
f_Th900 = []
f_Th850 = []
f_Th700 = []
f_Th500 = []
f_slp = []
f_sst = []
f_The900 = []
f_The850 = []
f_The700 = []
f_The500 = []
f_T500 = []
f_T700 = []
f_avo = []
f_pvo = []
f_wind10 = []
mcao700 =[]
mcao500= []
f_Th500_sst = []
f_Th700_sst = []
f_Th900_sst = []
f_wind_t = []
f_hfx = []
f_lh = []

In [ ]:
for file in filename:
    # чтение файлов
    df = nc.Dataset(file,engine='netcdf4')
    name_file = file[29:39]
    
    if file[34:36] not in ['01', '02', '03', '04', '05', '10', '11', '12']:
        df.close()
        continue
    
    # чтение нужных переменных
    theta = wrf.getvar(df, 'th', timeidx=wrf.ALL_TIMES)[:]
    wind10_u = wrf.getvar(df, 'U10', timeidx=wrf.ALL_TIMES)[:]
    wind10_v = wrf.getvar(df, 'V10', timeidx=wrf.ALL_TIMES)[:]
    wind_v = wrf.getvar(df, 'va', timeidx=wrf.ALL_TIMES)[:]
    wind_u = wrf.getvar(df, 'ua', timeidx=wrf.ALL_TIMES)[:]
    slp = wrf.getvar(df, 'slp', timeidx=wrf.ALL_TIMES)[:]
    sst = wrf.getvar(df, 'SST',timeidx=wrf.ALL_TIMES)[:]
    tt = wrf.getvar(df, 'temp', timeidx=wrf.ALL_TIMES)[:]
    avo = wrf.getvar(df, 'avo', timeidx=wrf.ALL_TIMES)[:]
    pvo = wrf.getvar(df, 'pvo',timeidx=wrf.ALL_TIMES)[:]
    u_t = wrf.vinterp(df, wind_u, vert_coord='pressure',interp_levels = [400],meta=False,timeidx=wrf.ALL_TIMES)
    v_t = wrf.vinterp(df, wind_v, vert_coord='pressure',interp_levels = [400],meta=False,timeidx=wrf.ALL_TIMES)
    hfx = wrf.getvar(df, 'HFX', timeidx=wrf.ALL_TIMES)[:] 
    lh = wrf.getvar(df, 'LH', timeidx=wrf.ALL_TIMES)[:] 
    wind_t = np.sqrt(v_t**2 + u_t**2)
    
# расчет новых переменных
    Th900 = wrf.vinterp(df,theta,vert_coord='pressure', interp_levels=[900],meta=False, timeidx=wrf.ALL_TIMES)
    Th850 = wrf.vinterp(df,theta,vert_coord='pressure', interp_levels=[850],meta=False,timeidx=wrf.ALL_TIMES)
    Th700 = wrf.vinterp(df, theta, vert_coord='pressure',interp_levels = [700],meta=False,timeidx=wrf.ALL_TIMES)
    Th500 = wrf.vinterp(df, theta, vert_coord='pressure',interp_levels = [500],meta=False,timeidx=wrf.ALL_TIMES)
    wind10 = np.sqrt(wind10_u**2 + wind10_v**2)
    u_t = wrf.vinterp(df, wind_u, vert_coord='pressure',interp_levels = [850],meta=False,timeidx=wrf.ALL_TIMES)
    v_t = wrf.vinterp(df, wind_v, vert_coord='pressure',interp_levels = [850],meta=False,timeidx=wrf.ALL_TIMES)
    wind_t = np.sqrt(v_t**2 + u_t**2)
    T500 =  wrf.vinterp(df, tt, vert_coord='pressure',interp_levels = [500],meta=False,timeidx=wrf.ALL_TIMES)
    T700 =  wrf.vinterp(df, tt, vert_coord='pressure',interp_levels = [700],meta=False,timeidx=wrf.ALL_TIMES)
    Thsst = sst*((1000/slp)**(287/1004))
    Thsst1 = np.reshape(Thsst.to_numpy(), (8,1,550,550))
    avo850 = wrf.vinterp(df,avo,vert_coord='pressure', interp_levels=[850],meta=False, timeidx=wrf.ALL_TIMES)
    pvo400 = wrf.vinterp(df,pvo,vert_coord='pressure', interp_levels=[400],meta=False, timeidx=wrf.ALL_TIMES)
    
    # расчет финальных переменных 
    f_T500.append((sst - T500[:,0,:,:]))
    f_T700.append((sst - T700[:,0,:,:]))
    f_Th500.append((Thsst - Th500[:,0,:,:]))
    f_Th700.append((Thsst - Th700[:,0,:,:]))
    f_Th850.append((Thsst - Th850[:,0,:,:]))
#     f_Th500_sst.append((Th500[:,0,:,:]-sst))
    f_Th700_sst.append((Th700[:,0,:,:]-sst))
    f_Th900_sst.append((Th900[:,0,:,:]-sst))
    f_wind_t.append(wind_t)
    mcao700.append(((7*(10**5)/700)*(np.log(Thsst)-np.log(Th700[:,0,:,:]))))
    mcao500.append(((7*(10**5)/500)*(np.log(Thsst)-np.log(Th500[:,0,:,:]))))
    f_wind10.append(wind10)
    f_slp.append(slp)
    f_sst.append(sst)
    f_avo.append(avo850)
    f_pvo.append(pvo400)
    f_wind_t.append(wind_t)
    f_hfx.append(hfx)
    f_lh.append(lh)

    df.close()

In [ ]:
np.save(file = f'SST50.npy', arr = f_T500)
np.save(file = f'SST700_.npy', arr = f_T700)
np.save(file = f'Th500.npy', arr = f_Th500)
np.save(file = f'Th700.npy', arr = f_Th700)
np.save(file = f'Th850.npy', arr = f_Th850)
np.save(file = f'wind10.npy', arr = f_wind10)
np.save(file = f'avo850.npy', arr = f_avo)
np.save(file = f'slp.npy', arr = f_slp)
np.save(file = f'sst.npy', arr = f_sst)
np.save(file = f'mcao700.npy', arr = mcao700)
np.save(file = f'mcao500.npy', arr = mcao500)
np.save(file = f'pvo400.npy', arr = f_pvo)
np.save(file = f'Th500_sst.npy', arr = f_Th500_sst)
np.save(file = f'Th700_sst.npy', arr = f_Th700_sst)
np.save(file = f'Th900_sst.npy', arr = f_Th900_sst)
np.save(file = f'wind_t.npy', arr = f_wind_t)
np.save('lh_2006.npy', arr=f_lh)
np.save('hfx_2006.npy', arr=f_hfx)